In [4]:
! pip install earthengine-api geemap

^C


  Using cached earthengine_api-1.5.5-py3-none-any.whl.metadata (2.1 kB)
  Using cached geemap-0.35.3-py2.py3-none-any.whl.metadata (12 kB)
  Using cached google_cloud_storage-3.1.0-py2.py3-none-any.whl.metadata (12 kB)
  Using cached google_api_python_client-2.163.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached bqplot-0.12.44-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached eerepr-0.1.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached folium-0.19.5-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ipyevents-2.0.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached ipyfilechooser-0.6.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached ipyleaflet-0.19.2-py3-none-any.whl.metadata (5.3 kB)
  Using cached ipytree-0.2.2-py2.py3-none-any.whl.metadata (849 by


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import ee
import geemap

# Authenticate and initialize
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [11]:
import pandas as pd
import ee
import datetime

# Authenticate & Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load historical flood data
df_historical = pd.read_csv('./datasets/floods_inventory/info.csv')

# Filter data to include only floods after 2015
# df_historical['Start Date'] = pd.to_datetime(df_historical['Start Date'], errors='coerce')
# df_historical['End Date'] = pd.to_datetime(df_historical['End Date'], errors='coerce')
# df_historical = df_historical[df_historical['Start Date'].dt.year >= 2015]
df_filter = df_historical[df_historical['Start Date'].str.contains('2015|2016|2017|2018|2019|2020|2021|2022|2023')]
# Define buffer size (e.g., 50 km)
buffer_size = 50000  # 50 km

def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def get_s2_image(lat, lon, date, buffer_size_m=50000):
    # Create point and buffer it
    point = ee.Geometry.Point(lon, lat)
    region = point.buffer(buffer_size_m).bounds()

    # Define date range
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')  # one day window

    # Load and filter collection
    collection = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(start_date, end_date)
        .filterBounds(region)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
    )

    # Get the first image (or median if needed)
    image = collection.first().clip(region)
    return image.select(['B2', 'B3', 'B4', 'B8', 'B11'])  # Blue, Green, Red, NIR, SWIR


# # Cloud Mask Function (QA60-based)
# def mask_s2_clouds(image):
#     qa = image.select("QA60")  # Sentinel-2 cloud mask band

#     # Bits 10 and 11 are clouds and cirrus, respectively
#     cloud_bit_mask = 1 << 10
#     cirrus_bit_mask = 1 << 11

#     # Both flags should be zero for clear pixels
#     mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
#         qa.bitwiseAnd(cirrus_bit_mask).eq(0)
#     )

#     # Apply mask and scale the image
#     return image.updateMask(mask).divide(10000)

# # Loop through each row and extract imagery
# for index, row in df_historical.iterrows():
#     try:
#         lat, lon = row['Latitude'], row['Longitude']
        
#         # Validate coordinates
#         if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
#             print(f"Invalid coordinates: ({lat}, {lon}) at index {index}. Skipping...")
#             continue
            
#         start_date, end_date = row['Start Date'], row['End Date']

#         # Ensure valid date range
#         if start_date >= end_date:
#             print(f"Invalid date range at index {index}: {start_date} to {end_date}. Swapping dates.")
#             start_date, end_date = end_date, start_date

#         # Ensure at least one day difference
#         if (end_date - start_date).days < 1:
#             print(f"Date range too short at index {index}. Extending end date by 1 day.")
#             end_date = start_date + datetime.timedelta(days=1)

#         # Convert dates to string format for GEE
#         start_date_str = start_date.strftime('%Y-%m-%d')
#         end_date_str = end_date.strftime('%Y-%m-%d')

#         # Define AOI (point with buffer)
#         poi = ee.Geometry.Point([lon, lat])
#         aoi = poi.buffer(buffer_size)

#         # Load Sentinel-2 Harmonized data with cloud masking
#         s2_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
#                          .filterBounds(aoi)
#                          .filterDate(start_date_str, end_date_str)
#                          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Stricter cloud filtering
#                          .map(mask_s2_clouds)  # Apply cloud mask
#                          .sort("CLOUDY_PIXEL_PERCENTAGE"))

#         # Check if images are available
#         count = s2_collection.size().getInfo()
#         print(f"Found {count} images for location ({lat}, {lon}) from {start_date_str} to {end_date_str}")

#         if count == 0:
#             print(f"No suitable images found for location ({lat}, {lon}). Skipping...")
#             continue

#         # Select the first image from the collection
#         image = s2_collection.median().clip(aoi)

#         # Visualization Parameters
#         rgb_vis = {
#             'min': 0.0,
#             'max': 0.3,
#             'bands': ['B4', 'B3', 'B2'],  # True Color Composite
#         }

#         # Define export task
#         task = ee.batch.Export.image.toDrive(
#     image=image,
#     description=f"Sentinel2_Flood_{index}",
#     folder="GEE_Flood_Images",
#     fileNamePrefix=f"Sentinel2_Flood_{index}_{lat}_{lon}",
#     scale=10,
#     region=aoi,
#     fileFormat='GeoTIFF',
#     maxPixels=1e9  # Increase max allowed pixels
# )


#         # Start the export task
#         task.start()
#         print(f"Exporting {index} to Google Drive...")

#     except Exception as e:
#         print(f"Error processing location ({lat}, {lon}): {str(e)}")

# print("All export tasks initiated! Check Google Drive for results.")


Found 0 images for location (26.2074, 82.6165) from 2015-07-15 to 2015-08-19
No suitable images found for location (26.2074, 82.6165). Skipping...
Found 0 images for location (26.8946, 93.751) from 2015-08-13 to 2015-11-09
No suitable images found for location (26.8946, 93.751). Skipping...
Invalid date range at index 2: 2015-10-11 00:00:00 to 2015-04-12 00:00:00. Swapping dates.
Found 7 images for location (11.8278, 78.8554) from 2015-04-12 to 2015-10-11
Exporting 2 to Google Drive...
Invalid date range at index 3: 2016-04-20 00:00:00 to 2016-01-05 00:00:00. Swapping dates.
Found 22 images for location (27.464, 95.6068) from 2016-01-05 to 2016-04-20
Exporting 3 to Google Drive...
Found 2 images for location (27.068, 93.949) from 2016-06-29 to 2016-08-26
Exporting 4 to Google Drive...
Invalid date range at index 5: 2016-07-07 00:00:00 to 2016-03-08 00:00:00. Swapping dates.
Found 26 images for location (22.8107, 80.8349) from 2016-03-08 to 2016-07-07
Exporting 5 to Google Drive...
Foun

In [14]:
#  os
# import re  # Import regex module

# # Initialize a list to store extracted data
# satellite_features = []

# for tiff in geo_tiffs:
#     with rasterio.open(tiff) as src:
#         image_array = src.read()  # Read all bands

#         # Sentinel-2 Band Mapping
#         B4 = image_array[3]  # Red
#         B3 = image_array[2]  # Green
#         B8 = image_array[7]  # NIR
#         B11 = image_array[10] # SWIRimport

#         # Compute indices
#         ndvi = np.nanmean(calculate_ndvi(B8, B4))
#         ndwi = np.nanmean(calculate_ndwi(B3, B8))
#         ndbi = np.nanmean(calculate_ndbi(B11, B8))

#         # Extract metadata from filename
#         filename = os.path.basename(tiff)  # Get filename only
#         metadata = filename.replace(".tif", "").split("_")  # Remove extension and split

#         try:
#             # Extract latitude from metadata[5]
#             lat = float(metadata[5])

#             # Extract longitude: Remove the extra `-000000XXXX` part using regex
#             lon = float(re.split(r'-', metadata[6])[0])  # Take only the first number

#         except ValueError:
#             print(f"Skipping file {filename} due to incorrect metadata format.")
#             continue

#         # Store extracted features
#         satellite_features.append({
#             "Latitude": lat,
#             "Longitude": lon,
#             "NDVI": ndvi,
#             "NDWI": ndwi,
#             "NDBI": ndbi
#         })

# # Convert to DataFrame
# df_satellite = pd.DataFrame(satellite_features)

# # Save extracted features
# df_satellite.to_csv("new_extracted_satellite_features.csv", index=False)

# print("Satellite features extracted successfully!")


ValueError: could not convert string to float: 'Flood'

In [17]:
# # Print the filenames to check the structure
# print(geo_tiffs[:5])  # Print first 5 filenames

# for tiff in geo_tiffs[:5]:  # Print for first 5 files
#     print(tiff.split("_"))



['./datasets/new_satellite_imagery\\Sentinel2_Flood_12_28.1657_79.076-0000000000-0000000000.tif', './datasets/new_satellite_imagery\\Sentinel2_Flood_12_28.1657_79.076-0000000000-0000007680.tif', './datasets/new_satellite_imagery\\Sentinel2_Flood_12_28.1657_79.076-0000007680-0000000000.tif', './datasets/new_satellite_imagery\\Sentinel2_Flood_12_28.1657_79.076-0000007680-0000007680.tif', './datasets/new_satellite_imagery\\Sentinel2_Flood_13_10.489_79.0039-0000000000-0000000000.tif']
['./datasets/new', 'satellite', 'imagery\\Sentinel2', 'Flood', '12', '28.1657', '79.076-0000000000-0000000000.tif']
['./datasets/new', 'satellite', 'imagery\\Sentinel2', 'Flood', '12', '28.1657', '79.076-0000000000-0000007680.tif']
['./datasets/new', 'satellite', 'imagery\\Sentinel2', 'Flood', '12', '28.1657', '79.076-0000007680-0000000000.tif']
['./datasets/new', 'satellite', 'imagery\\Sentinel2', 'Flood', '12', '28.1657', '79.076-0000007680-0000007680.tif']
['./datasets/new', 'satellite', 'imagery\\Sentinel

In [41]:
import pandas as pd
import ee
import datetime

# Authenticate & Initialize Earth Engine
# ee.Authenticate()
# ee.Initialize()

# Load historical flood data
df_historical = pd.read_csv('./datasets/floods_inventory/India_Floods_Inventory.csv')

# Filter data to include only floods after 2015
# df_historical['Start Date'] = pd.to_datetime(df_historical['Start Date'], errors='coerce')
# df_historical['End Date'] = pd.to_datetime(df_historical['End Date'], errors='coerce')
# df_historical = df_historical[df_historical['Start Date'].dt.year >= 2015]
df_filter = df_historical[df_historical['Start Date'].str.contains('2015|2016|2017|2018|2019|2020|2021|2022|2023')]
df_filter.dropna(subset=['Latitude', 'Longitude'], inplace=True)  # Drop rows with NaN coordinates
df_filter.reset_index(drop=True, inplace=True)  # Reset index after dropping rows
df_historical=df_filter[['Latitude', 'Longitude', 'Start Date', 'End Date']]
df_historical.to_csv('./datasets/floods_inventory/info.csv', index=False)  # Save filtered data

df_factors = pd.read_csv('./datasets/india_flood_risk/flood_risk_dataset_india.csv')
#make a csv of overlapping data from df_historical and df_factors based on coordinates
import numpy as np
from scipy.spatial import cKDTree

# Load your datasets
df_hist = pd.read_csv('./datasets/floods_inventory/info.csv')
df_fact = pd.read_csv('./datasets/india_flood_risk/flood_risk_dataset_india.csv')

# Extract coordinates
coords_hist = df_hist[['Latitude', 'Longitude']].to_numpy()
coords_fact = df_fact[['Latitude', 'Longitude']].to_numpy()

# Build KDTree on df_factors
tree = cKDTree(coords_fact)

# Define a distance threshold (in degrees)
# ~0.01 degrees ≈ 1.1 km (very rough approximation)
threshold = 0.5

# Query tree for all points in df_hist within threshold
distances, indices = tree.query(coords_hist, distance_upper_bound=threshold)

# Filter matches where distance is within threshold
valid = distances != np.inf
matched_hist = df_hist[valid].reset_index(drop=True)
matched_fact = df_fact.iloc[indices[valid]].reset_index(drop=True)

# Combine matched points (optional)
matched = pd.concat([matched_hist, matched_fact], axis=1)

print(matched.head())
print(f"Found {len(matched)} approximate matches.")


C:\Users\hp\AppData\Local\Temp\ipykernel_27052\2566850716.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.dropna(subset=['Latitude', 'Longitude'], inplace=True)  # Drop rows with NaN coordinates


   Latitude  Longitude  Start Date    End Date   Latitude  Longitude  \
0   33.1767    76.4058  2015-03-20  2015-03-31  33.171409  76.447781   
1   10.7121    79.1771  2015-05-17  2015-05-21  10.433041  78.962892   
2   26.9195    93.8747  2015-02-06  2015-06-29  26.835759  93.760925   
3   21.8734    70.7643  2015-06-24  2015-06-29  21.662200  70.595955   
4   26.2074    82.6165  2015-07-15  2015-08-19  26.177276  82.582468   

   Rainfall (mm)  Temperature (°C)  Humidity (%)  River Discharge (m³/s)  \
0     131.044676         43.204075     83.279018             1114.604526   
1      22.025647         21.785839     84.696733             2718.366764   
2     173.153159         38.875367     97.630651             2795.331123   
3      47.477868         31.774465     64.771080             1504.834080   
4     217.957042         26.952350     28.458914             1788.846557   

   Water Level (m)  Elevation (m)  Land Cover Soil Type  Population Density  \
0         2.163148    3870.9485

In [42]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Load your datasets
df_hist = pd.read_csv('./datasets/floods_inventory/info.csv')
df_fact = pd.read_csv('./datasets/india_flood_risk/flood_risk_dataset_india.csv')

# Extract coordinates
coords_hist = df_hist[['Latitude', 'Longitude']].to_numpy()
coords_fact = df_fact[['Latitude', 'Longitude']].to_numpy()

# Build KDTree on df_factors
tree = cKDTree(coords_fact)

# Define a distance threshold (in degrees)
# ~0.01 degrees ≈ 1.1 km (very rough approximation)
threshold = 0.5

# Query tree for all points in df_hist within threshold
distances, indices = tree.query(coords_hist, distance_upper_bound=threshold)

# Filter matches where distance is within threshold
valid = distances != np.inf
matched_hist = df_hist[valid].reset_index(drop=True)
matched_fact = df_fact.iloc[indices[valid]].reset_index(drop=True)

# Combine matched points (optional)
matched = pd.concat([matched_hist, matched_fact], axis=1)

print(matched.head())
print(f"Found {len(matched)} approximate matches.")
#save to csv
matched.to_csv('./datasets/overlapping_data.csv', index=False)  # Save filtered data


   Latitude  Longitude  Start Date    End Date   Latitude  Longitude  \
0   33.1767    76.4058  2015-03-20  2015-03-31  33.171409  76.447781   
1   10.7121    79.1771  2015-05-17  2015-05-21  10.433041  78.962892   
2   26.9195    93.8747  2015-02-06  2015-06-29  26.835759  93.760925   
3   21.8734    70.7643  2015-06-24  2015-06-29  21.662200  70.595955   
4   26.2074    82.6165  2015-07-15  2015-08-19  26.177276  82.582468   

   Rainfall (mm)  Temperature (°C)  Humidity (%)  River Discharge (m³/s)  \
0     131.044676         43.204075     83.279018             1114.604526   
1      22.025647         21.785839     84.696733             2718.366764   
2     173.153159         38.875367     97.630651             2795.331123   
3      47.477868         31.774465     64.771080             1504.834080   
4     217.957042         26.952350     28.458914             1788.846557   

   Water Level (m)  Elevation (m)  Land Cover Soil Type  Population Density  \
0         2.163148    3870.9485

In [6]:
import pandas as pd
import ee
import datetime

# Initialize Earth Engine
ee.Initialize()

# Load historical flood data
df_historical = pd.read_csv('./datasets/floods_inventory/info_dont_use.csv')
df_historical.dropna(subset=['Latitude', 'Longitude'], inplace=True)  # Drop rows with NaN coordinates
df_historical.reset_index(drop=True, inplace=True)  # Reset index after dropping rows

# Filter data to include only floods after 2015
df_historical['Start Date'] = pd.to_datetime(df_historical['Start Date'], errors='coerce')
df_historical['End Date'] = pd.to_datetime(df_historical['End Date'], errors='coerce')
df_historical = df_historical[df_historical['Start Date'].dt.year >= 2015]

# Define buffer size (e.g., 50 km)
buffer_size = 50000  # 50 km

# Cloud Mask Function (QA60-based)
def mask_s2_clouds(image):
    qa = image.select("QA60")  # Sentinel-2 cloud mask band

    # Bits 10 and 11 are clouds and cirrus, respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be zero for clear pixels
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )

    # Apply mask and scale the image
    return image.updateMask(mask).divide(10000)

# Loop through each row and extract imagery
for index, row in df_historical.iterrows():
    try:
        lat, lon = row['Latitude'], row['Longitude']
        
        # Validate coordinates
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            print(f"Invalid coordinates: ({lat}, {lon}) at index {index}. Skipping...")
            continue
            
        start_date = row['Start Date']
        
        # Calculate t-7 date (7 days before start date)
        t_minus_7_date = start_date - datetime.timedelta(days=7)

        # Convert dates to string format for GEE
        t_minus_7_date_str = t_minus_7_date.strftime('%Y-%m-%d')
        start_date_str = start_date.strftime('%Y-%m-%d')

        # Define AOI (point with buffer)
        poi = ee.Geometry.Point([lon, lat])
        aoi = poi.buffer(buffer_size)

        # Load Sentinel-2 Harmonized data with cloud masking
        s2_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                         .filterBounds(aoi)
                         .filterDate(t_minus_7_date_str, start_date_str)
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Stricter cloud filtering
                         .map(mask_s2_clouds)  # Apply cloud mask
                         .sort("CLOUDY_PIXEL_PERCENTAGE"))

        # Check if images are available
        count = s2_collection.size().getInfo()
        print(f"Found {count} images for location ({lat}, {lon}) from {t_minus_7_date_str} to {start_date_str}")

        if count == 0:
            print(f"No suitable images found for location ({lat}, {lon}). Skipping...")
            continue

        # Select the median image from the collection
        image = s2_collection.median().clip(aoi)

        # Visualization Parameters
        rgb_vis = {
            'min': 0.0,
            'max': 0.3,
            'bands': ['B4', 'B3', 'B2'],  # True Color Composite
        }

        # Define export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"PreFlood_t-7_to_t-0_{index}",
            folder="GEE_PreFlood_Images",
            fileNamePrefix=f"PreFlood_t-7_to_t-0_{index}_{lat}_{lon}",
            scale=10,
            region=aoi,
            fileFormat='GeoTIFF',
            maxPixels=1e9  # Increase max allowed pixels
        )

        # Start the export task
        task.start()
        print(f"Exporting pre-flood imagery {index} to Google Drive...")

    except Exception as e:
        print(f"Error processing location ({lat}, {lon}): {str(e)}")

print("All export tasks initiated! Check Google Drive for results.") 



Found 0 images for location (26.2074, 82.6165) from 2015-07-08 to 2015-07-15
No suitable images found for location (26.2074, 82.6165). Skipping...
Found 0 images for location (26.8946, 93.751) from 2015-08-06 to 2015-08-13
No suitable images found for location (26.8946, 93.751). Skipping...
Found 0 images for location (11.8278, 78.8554) from 2015-10-04 to 2015-10-11
No suitable images found for location (11.8278, 78.8554). Skipping...
Found 0 images for location (27.464, 95.6068) from 2016-04-13 to 2016-04-20
No suitable images found for location (27.464, 95.6068). Skipping...
Found 0 images for location (27.068, 93.949) from 2016-06-22 to 2016-06-29
No suitable images found for location (27.068, 93.949). Skipping...
Found 0 images for location (22.8107, 80.8349) from 2016-06-30 to 2016-07-07
No suitable images found for location (22.8107, 80.8349). Skipping...
Found 4 images for location (21.7477, 73.4119) from 2016-01-01 to 2016-01-08
Exporting pre-flood imagery 6 to Google Drive...


In [5]:
ee.Authenticate()


Successfully saved authorization token.


In [8]:
import pandas as pd
import ee
import datetime
import random

# Initialize Earth Engine
ee.Initialize()

# Load flood inventory data
df_historical = pd.read_csv('./datasets/floods_inventory/info_dont_use.csv')
df_historical.dropna(subset=['Latitude', 'Longitude'], inplace=True)
df_historical.reset_index(drop=True, inplace=True)

# Cloud Mask Function
def mask_s2_clouds(image):
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Constants
buffer_size = 50000  # 50 km buffer
start_year = 2022
end_year = 2024
export_folder = "GEE_NonFlood_Images"

# Loop through each coordinate and pick a random week after 2022
for index, row in df_historical.iterrows():
    try:
        lat, lon = row['Latitude'], row['Longitude']
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            print(f"Invalid coordinates: ({lat}, {lon}) at index {index}. Skipping...")
            continue

        # Generate random date after 2022
        random_year = random.randint(start_year, end_year)
        random_month = random.randint(1, 12)
        random_day = random.randint(1, 21)  # So that t+7 doesn't exceed month end

        random_date = datetime.date(random_year, random_month, random_day)
        random_date_plus_7 = random_date + datetime.timedelta(days=7)

        start_date_str = random_date.strftime('%Y-%m-%d')
        end_date_str = random_date_plus_7.strftime('%Y-%m-%d')

        # Area of Interest
        poi = ee.Geometry.Point([lon, lat])
        aoi = poi.buffer(buffer_size)

        # Sentinel-2 data with filters
        s2_collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                         .filterBounds(aoi)
                         .filterDate(start_date_str, end_date_str)
                         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                         .map(mask_s2_clouds)
                         .sort("CLOUDY_PIXEL_PERCENTAGE"))

        count = s2_collection.size().getInfo()
        print(f"[{index}] Found {count} non-flood images from {start_date_str} to {end_date_str} at ({lat}, {lon})")

        if count == 0:
            print(f"[{index}] No non-flood images available. Skipping.")
            continue

        # Select median image
        image = s2_collection.median().clip(aoi)

        # Export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"NonFlood_{index}",
            folder=export_folder,
            fileNamePrefix=f"NonFlood_{index}_{lat}_{lon}",
            scale=10,
            region=aoi,
            fileFormat='GeoTIFF',
            maxPixels=1e9
        )
        task.start()
        print(f"[{index}] Exporting non-flood image to Drive...")

    except Exception as e:
        print(f"[{index}] Error processing location ({lat}, {lon}): {str(e)}")

print("✅ All non-flood export tasks initiated! Check Google Drive for results.")


[0] Found 2 non-flood images from 2024-03-16 to 2024-03-23 at (26.2074, 82.6165)
[0] Exporting non-flood image to Drive...
[1] Found 4 non-flood images from 2023-11-16 to 2023-11-23 at (26.8946, 93.751)
[1] Exporting non-flood image to Drive...
[2] Found 4 non-flood images from 2024-09-12 to 2024-09-19 at (11.8278, 78.8554)
[2] Exporting non-flood image to Drive...
[3] Found 2 non-flood images from 2023-07-15 to 2023-07-22 at (27.464, 95.6068)
[3] Exporting non-flood image to Drive...
[4] Found 0 non-flood images from 2022-05-19 to 2022-05-26 at (27.068, 93.949)
[4] No non-flood images available. Skipping.
[5] Found 7 non-flood images from 2023-02-18 to 2023-02-25 at (22.8107, 80.8349)
[5] Exporting non-flood image to Drive...
[6] Found 0 non-flood images from 2022-08-11 to 2022-08-18 at (21.7477, 73.4119)
[6] No non-flood images available. Skipping.
[7] Found 6 non-flood images from 2023-01-11 to 2023-01-18 at (25.6099, 85.1898)
[7] Exporting non-flood image to Drive...
[8] Found 0 no

In [12]:
pip install rasterio

  Obtaining dependency information for rasterio from https://files.pythonhosted.org/packages/7e/1f/56462740694de764fde264051224fcbf800dad43cac92a66753153128866/rasterio-1.4.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for affine from https://files.pythonhosted.org/packages/0b/f7/85273299ab57117850cc0a936c64151171fac4da49bc6fba0dad984a7c5f/affine-2.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for cligj>=0.5 from https://files.pythonhosted.org/packages/73/86/43fa9f15c5b9fb6e82620428827cd3c284aa933431405d1bcf5231ae3d3e/cligj-0.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for click-plugins from https://files.pythonhosted.org/packages/e9/da/824b92d9942f4e472702488857914bdd50f73021efea15b4cad9aca8ecef/click_plugins-1.1.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
    --------------------------------------- 0.4/25.5 MB 8.9 MB/s eta 0:00:03
   -- -------------------------